In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision import models
from torchvision.models import ResNet18_Weights
resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)
from torch.utils.data import Dataset, DataLoader

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
IMAGE_DIR = r"C:\Users\DELL\Downloads\Apple Leaf Diseases Image Dataset of ICAR-CITH\Apple Leaf Diseases Image Dataset of ICAR-CITH\Apple Diseases Images\Apple Diseases Images\Apple Apple scab"


cuda


In [3]:
class AppleScabDataset(Dataset):
    def __init__(self, img_dir, transform):
        self.img_dir = img_dir
        self.images = [
            f for f in os.listdir(img_dir)
            if f.lower().endswith(('.jpg','.png','.jpeg'))
        ]
        self.transform = transform

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        view1 = self.transform(img)
        view2 = self.transform(img)

        return view1, view2

    def __len__(self):
        return len(self.images)


In [4]:
ssl_transform = T.Compose([
    T.ToPILImage(),
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.RandomApply([
        T.ColorJitter(0.4, 0.4, 0.4, 0.1)
    ], p=0.8),
    T.ToTensor()
])


In [5]:
dataset = AppleScabDataset(IMAGE_DIR, ssl_transform)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

class SSLModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = models.resnet18(pretrained=True)
        self.encoder.fc = nn.Identity()
        self.projector = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 128)
        )

    def forward(self, x):
        h = self.encoder(x)
        z = self.projector(h)
        return z

model = SSLModel().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


C:\Users\DELL\anaconda3\envs\yolov8_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DELL\anaconda3\envs\yolov8_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
def train_ssl(model, loader, optimizer, epochs=5):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0

        for v1, v2 in loader:
            v1 = v1.to(DEVICE)
            v2 = v2.to(DEVICE)

            z1 = model(v1)
            z2 = model(v2)

            loss = contrastive_loss(z1, z2)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss:.4f}")
        
import torch.nn.functional as F

def contrastive_loss(z1, z2, temperature=0.5):
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)

    logits = torch.matmul(z1, z2.T) / temperature
    labels = torch.arange(z1.size(0)).to(z1.device)

    return nn.CrossEntropyLoss()(logits, labels)


In [9]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

model.eval()

import random

files = [
    f for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
]

img_name = random.choice(files)
img_path = os.path.join(IMAGE_DIR, img_name)

print(f"Randomly selected image: {img_name}")

# ---- IMAGE PREPARATION ----
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (224, 224))

img_tensor = T.ToTensor()(img_resized).unsqueeze(0).to(DEVICE)

# ---- GRAD-CAM ----
target_layer = model.encoder.layer4[-1]
cam = GradCAM(model=model, target_layers=[target_layer])

grayscale_cam = cam(input_tensor=img_tensor)[0]

visualization = show_cam_on_image(
    img_resized.astype("float32") / 255.0,
    grayscale_cam,
    use_rgb=True
)

cv2.imshow("Grad-CAM Lesion Localization", visualization)
cv2.waitKey(0)
cv2.destroyAllWindows()



Randomly selected image: 7ed79254-3e04-4e08-9c80-088cd9d7d8c9___FREC_Scab 3310_270deg.JPG
